In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [93]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn import metrics

In [94]:
df = pd.read_csv("data/LoanApprovalPrediction_cleaned.csv")
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0.0,0,0,55849,0.0,144.968804,360.0,1.0,2,1
1,1,1,1.0,0,0,54583,1508.0,128.000000,360.0,1.0,0,0
2,1,1,0.0,0,1,53000,0.0,66.000000,360.0,1.0,2,1
3,1,1,0.0,1,0,52583,2358.0,120.000000,360.0,1.0,2,1
4,1,0,0.0,0,0,56000,0.0,141.000000,360.0,1.0,2,1


In [95]:
from sklearn.preprocessing import StandardScaler

# List of columns to scale
scale_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']

# Instantiate scaler
scaler = StandardScaler()

# Fit and transform only those columns
df[scale_cols] = scaler.fit_transform(df[scale_cols])


In [96]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0.0,0,0,0.095951,-0.552892,-3.501564e-16,0.280854,1.0,2,1
1,1,1,1.0,0,0,-0.122234,-0.041852,-2.090562e-01,0.280854,1.0,0,0
2,1,1,0.0,0,1,-0.395052,-0.552892,-9.728983e-01,0.280854,1.0,2,1
3,1,1,0.0,1,0,-0.466919,0.246201,-3.076165e-01,0.280854,1.0,2,1
4,1,0,0.0,0,0,0.121975,-0.552892,-4.889580e-02,0.280854,1.0,2,1


In [97]:
X = df.drop('Loan_Status', axis=1)
Y = df['Loan_Status']

In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



In [99]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((478, 11), (120, 11), (478,), (120,))

In [118]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn import metrics

knn = KNeighborsClassifier(n_neighbors=3)
rfc = RandomForestClassifier(n_estimators = 7,
                             criterion = 'entropy',
                             random_state =7)
svc = SVC()
lc = LogisticRegression(max_iter=1000)
xgc = XGBClassifier( eval_metric='logloss',
    max_depth=3,             # limit tree depth (default is 6)
    learning_rate=0.2,       # slower learning
    n_estimators=180,        # total trees
    subsample=0.8,           # use 80% of data per tree
    colsample_bytree=0.8,    # use 80% of features per tree
    reg_alpha=0.3,           # L1 regularization
    reg_lambda=1,            # L2 regularization
    random_state=42)

# making predictions on the training set
for clf in (rfc, knn, svc,lc,xgc):
    clf.fit(X_train, y_train)
    Y_pred = clf.predict(X_train)
    print("Accuracy score of ",
          clf.__class__.__name__,
          "=",100*metrics.accuracy_score(y_train, 
                                         Y_pred))

Accuracy score of  RandomForestClassifier = 96.23430962343096
Accuracy score of  KNeighborsClassifier = 83.68200836820083
Accuracy score of  SVC = 82.63598326359832
Accuracy score of  LogisticRegression = 80.5439330543933
Accuracy score of  XGBClassifier = 96.44351464435147


In [119]:
# making predictions on the testing set
for clf in (rfc, knn, svc,lc,xgc):
    clf.fit(X_train, y_train)
    Y_pred = clf.predict(X_test)
    print("Accuracy score of ",
          clf.__class__.__name__,"=",
          100*metrics.accuracy_score(y_test,
                                     Y_pred))

Accuracy score of  RandomForestClassifier = 78.33333333333333
Accuracy score of  KNeighborsClassifier = 72.5
Accuracy score of  SVC = 80.83333333333333
Accuracy score of  LogisticRegression = 81.66666666666667
Accuracy score of  XGBClassifier = 83.33333333333334


In [120]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lc, X_train, y_train, cv=5)
print("Cross-Validation Accuracy (LogReg):", scores.mean())


Cross-Validation Accuracy (LogReg): 0.799232456140351


In [121]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = xgc.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[21 14]
 [ 6 79]]
              precision    recall  f1-score   support

           0       0.78      0.60      0.68        35
           1       0.85      0.93      0.89        85

    accuracy                           0.83       120
   macro avg       0.81      0.76      0.78       120
weighted avg       0.83      0.83      0.83       120

